<h1 align="center"> NLP (Natural Language Processing) on SMS Spam Detection </h1> <br>
<img src="https://media.giphy.com/media/Qx0zhgnni9e0g/giphy.gif"><br><br>

### **Before starting:  If you find this kernal helpful please  <font color="red"><b>UPVOTE</b></font>  the project. It would be highly appreciated.**
<br><br><br>



## The data
The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam.

For more info on the data, please check out: <a src="https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection"> UCI Machine Learning Repository</a>

<img src="https://media.giphy.com/media/xLhloTgdu7i92/giphy.gif"><br><br>



## Outline: <br>

I. [Loading data and other imports](#loading)<br>

II. [Exploratory Data Analysis](#EDA)<br>

III. [Data Visualization](#data_vi)<br>

IV. <b> Creating the Model</b><br>

V. <b>Evaluating Model Performance.</b><br>



## References:
1) <a src="https://www.udemy.com/course/python-for-data-science-and-machine-learning-bootcamp/"> Python for Data Science and Machine Learning Bootcamp </a> by  Jose Portilla <br> 





<a id="loading"></a>
## Loading the data and other imports

In [ ]:
import nltk 

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',delimiter=',',encoding='latin-1')

df.head()

In [ ]:
# change the column names;
spam_df=df[['v1','v2']]
spam_df.rename(columns={"v1": "label", "v2": "message"},inplace=True)


<a id="EDA"></a>
## EDA (Exploratory Data Analysis)

Let's check out some of the stats with some plots and the built-in methods in pandas!

In [ ]:
spam_df.describe()

Let's use **groupby** to use describe by label, this way we can begin to think about the features that separate ham and spam!

In [ ]:
spam_df.groupby('label').describe()

As we continue our analysis we want to start thinking about the features we are going to be using. This goes along with the general idea of [feature engineering](https://en.wikipedia.org/wiki/Feature_engineering). The better your domain knowledge on the data, the better your ability to engineer more features from it. Feature engineering is a very large part of spam detection in general. I encourage you to read up on the topic!

Let's make a new column to detect how long the text messages are:

In [ ]:
spam_df['length'] = spam_df['message'].apply(len)
spam_df.head()

<a id="data_vi"></a>
## Data Visualization
Let's visualize this! Let's do the imports:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
spam_df['length'].plot.hist(bins = 150)

In [ ]:
spam_df.describe()

From the histogram we can see some large length values. Using Describe method we could find out the max value. Let's find this large text message :
You can grab your data frame and spacify your condition to find it:

In [ ]:
spam_df[spam_df['length']==910]['message'].iloc[0]

Hmm, it is kind of wierd love letter! If you like go and read it!

But, we want to know that if the message length is distinguighing feature between spam and ham! We already see this outlier and we went ahead and explored all it.
let's go ahead and make a histogram of messages. 

In [ ]:
spam_df.hist(column='length',by='label',bins=60, figsize=(12,4)) 

What is very interesting here is just for a basic exploratory data analysis , we've been able to discover the trend that the spam messages tend to have more characters. Just visually it looks like  that length is very good feature to distinguish a spam message versus ham massage! 

## Text Preprocessing

Since our main issue for our data is that it's all in text format or strings. The classification algorithms need some sort of numerical feature vector in order to perform the classificaiton tasks. There are many actual methods where you can convert a corpus of stirngs to a vector format in the simplest approach is what is known as **bag of words** where each unique word in the text will be represented by one number. 

We are going to convert raw messages( a sequence of characters) into vectors( a sequence of numbers) and that's the first step we're going to write a function that will split a message into its individual words and return a list. We're also going to remove very common words the A if and etc, (aka stop words) and that's what we're going to be using the Teekay library. 

In [ ]:
import string 

We are also take advantage of pythons built in string library. The first thing we can do is removing punctuation. You can see it from below example;

In [ ]:
mess = 'Sample message! Notice: it has punctuation.'

In [ ]:
string.punctuation

Using list comperhension you can check if the above punctuation are in the message or not

In [ ]:
nopunc = [c for c in mess if c not in string.punctuation]

In [ ]:
nopunc

You see that there are no longer the punctuation in the message. In second step, let's see how we can remove stop words. We can import a list of English Stop words from an LPK.

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

These are such common words that when you're trying to do a spam or ham detection or any sort of classifcation of text data these words are so common that they're probably not going to be very helpful as far as trying to distinguigh one source of text form another. They are kind of useless to use as a feature. Let's test it on our sample example message:

In [ ]:
nopunc = ''.join(nopunc)
nopunc

In [ ]:
nopunc.split()

In [ ]:
clean_mess = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
clean_mess

 Now, we have the most important words. If you undestand it clearly now, let's write a function to perform it for our main datset.

In [ ]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

Now, if we check out our messages again , we have the message column. We're going to tokenize these messages (tokenization is just a term you use to describe the process of what we just did: converting a normal text string into a list of tokens and tokens are just the words we actually want that is a clean version of the words ) 

In [ ]:
spam_df.head()

In [ ]:
spam_df['message'].head(5).apply(text_process)

In above you see an example of tokinzation. You apply the process and function to the example. Basiclly what happens is we remove any stop words and we create a list of these tokens(the important ones that we want). 

Actully there are a lots of ways to continue to normalize the text data. We here have only done very simple text normalization in preprocessing and the Teekay library has lots of built in tools and great documentation on a lot of other methods of normalization for example stemming is a really common way to continue processing text data. Basically, it has a bunch of similar words such as 'ranning', 'ran', 'run'  , and stemming tries to break all of these down and retuns only 'run'. Teekay comes with a lot of dictionaries and built in data sets and corpuses and references for doing this porpuse.However, for our dataset stemming might not works well. The reason is that we have some abbrivated text messages , such as U, Nah, dont and etc. So let's skip this part now.

Now, let us focus back on vectorization. We have the messages as lists of tokens and now we need to convert each of those messages into a vector that sklearn algorithm models can work with. 

## Vectorization
Currently, we have the messages as lists of tokens (also known as [lemmas](http://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)) and now we need to convert each of those messages into a vector the SciKit Learn's algorithm models can work with.

Now we'll convert each message, represented as a list of tokens (lemmas) above, into a vector that machine learning models can understand.

We'll do that in three steps using the bag-of-words model:

1. Count how many times does a word occur in each message (Known as term frequency)

2. Weigh the counts, so that frequent tokens get lower weight (inverse document frequency)

3. Normalize the vectors to unit length, to abstract from the original text length (L2 norm)

Let's begin the first step:

Each vector will have as many dimensions as there are unique words in the SMS corpus.  We will first use SciKit Learn's **CountVectorizer**. This model will convert a collection of text documents to a matrix of token counts.

We can imagine this as a 2-Dimensional matrix. Where the 1-dimension is the entire vocabulary (1 row per word) and the other dimension are the actual documents, in this case a column per text message. 

For example:

<table border = “1“>
<tr>
<th></th> <th>Message 1</th> <th>Message 2</th> <th>...</th> <th>Message N</th> 
</tr>
<tr>
<td><b>Word 1 Count</b></td><td>0</td><td>1</td><td>...</td><td>0</td>
</tr>
<tr>
<td><b>Word 2 Count</b></td><td>0</td><td>0</td><td>...</td><td>0</td>
</tr>
<tr>
<td><b>...</b></td> <td>1</td><td>2</td><td>...</td><td>0</td>
</tr>
<tr>
<td><b>Word N Count</b></td> <td>0</td><td>1</td><td>...</td><td>1</td>
</tr>
</table>


Since there are so many messages, we can expect a lot of zero counts for the presence of that word in that document. Because of this, SciKit Learn will output a [Sparse Matrix](https://en.wikipedia.org/wiki/Sparse_matrix).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

There are a lot of arguments and parameters that can be passed to the CountVectorizer. In this case we will just specify the analyzer to be our own previously defined function:

In [ ]:
# Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_process).fit(spam_df['message'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

Let's take one text message and get its bag-of-words counts as a vector, putting to use our new bow_transformer:`

In [ ]:
message4 = spam_df['message'][3]
print(message4)

Now let's see its vector representation:

In [ ]:
bow4 = bow_transformer.transform([message4])
print(bow4)
print(bow4.shape)

This means that there are seven unique words in message number 4 (after removing common stop words). Two of them appear twice, the rest only once. Let's go ahead and check and confirm which ones appear twice:

In [ ]:
print(bow_transformer.get_feature_names()[3996])
print(bow_transformer.get_feature_names()[9445])

Now we can use .transform on our Bag-of-Words (bow) transformed object and transform the entire DataFrame of messages. Let's go ahead and check out how the bag-of-words counts for the entire SMS corpus is a large, sparse matrix:

In [ ]:
messages_bow = bow_transformer.transform(spam_df['message'])
print('Shape of Sparse Matrix: ', messages_bow.shape)
print('Amount of Non-Zero occurences: ', messages_bow.nnz)


In [ ]:
sparsity = (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

After the counting, the term weighting and normalization can be done with [TF-IDF](http://en.wikipedia.org/wiki/Tf%E2%80%93idf), using scikit-learn's `TfidfTransformer`.

____
### So what is TF-IDF?
TF-IDF stands for *term frequency-inverse document frequency*, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.

One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

**TF: Term Frequency**, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: 

*TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).*

**IDF: Inverse Document Frequency**, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: 

*IDF(t) = log_e(Total number of documents / Number of documents with term t in it).*

See below for a simple example.

**Example:**

Consider a document containing 100 words wherein the word cat appears 3 times. 

The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
____

Let's go ahead and see how we can do this in SciKit Learn:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

We'll go ahead and check what is the IDF (inverse document frequency) of the word `"u"` and of word `"university"`?

In [ ]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['u']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['university']])

To transform the entire bag-of-words corpus into TF-IDF corpus at once:

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

There are many ways the data can be preprocessed and vectorized. These steps involve feature engineering and building a "pipeline". I encourage you to check out SciKit Learn's documentation on dealing with text data as well as the expansive collection of available papers and books on the general topic of NLP.

## Training a model

With messages represented as vectors, we can finally train our spam/ham classifier. Now we can actually use almost any sort of classification algorithms. For a [variety of reasons](http://www.inf.ed.ac.uk/teaching/courses/inf2b/learnnotes/inf2b-learn-note07-2up.pdf), the Naive Bayes classifier algorithm is a good choice.
We'll be using scikit-learn here, choosing the [Naive Bayes](http://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier to start with:

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(messages_tfidf, spam_df['label'])

Let's try classifying our single random message and checking how we do:

In [ ]:
print('predicted:', spam_detect_model.predict(tfidf4)[0])
print('expected:', spam_df.label[3])

Fantastic! We've developed a model that can attempt to predict spam vs ham classification!

## Part 6: Model Evaluation
Now we want to determine how well our model will do overall on the entire dataset. Let's begin by getting all the predictions:

In [ ]:
all_predictions = spam_detect_model.predict(messages_tfidf)
print(all_predictions)

We can use SciKit Learn's built-in classification report, which returns [precision, recall,](https://en.wikipedia.org/wiki/Precision_and_recall) [f1-score](https://en.wikipedia.org/wiki/F1_score), and a column for support (meaning how many cases supported that classification). Check out the links for more detailed info on each of these metrics and the figure below:

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/700px-Precisionrecall.svg.png' width=400 />

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(spam_df['label'], all_predictions))

There are quite a few possible metrics for evaluating model performance. Which one is the most important depends on the task and the business effects of decisions based off of the model. For example, the cost of mis-predicting "spam" as "ham" is probably much lower than mis-predicting "ham" as "spam".

In the above "evaluation",we evaluated accuracy on the same data we used for training. **You should never actually evaluate on the same dataset you train on!**

Such evaluation tells us nothing about the true predictive power of our model. If we simply remembered each example during training, the accuracy on training data would trivially be 100%, even though we wouldn't be able to classify any new messages.

A proper way is to split the data into a training/test set, where the model only ever sees the **training data** during its model fitting and parameter tuning. The **test data** is never used in any way. This is then our final evaluation on test data is representative of true predictive performance.

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = \
train_test_split(spam_df['message'], spam_df['label'], test_size=0.2)

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

The test size is 20% of the entire dataset (1115 messages out of total 5572), and the training is the rest (4457 out of 5572). Note the default split would have been 30/70.

## Creating a Data Pipeline

Let's run our model again and then predict off the test set. We will use SciKit Learn's [pipeline](http://scikit-learn.org/stable/modules/pipeline.html) capabilities to store a pipeline of workflow. This will allow us to set up all the transformations that we will do to the data for future use. Let's see an example of how it works:

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

Now we can directly pass message text data and the pipeline will do our pre-processing for us! We can treat it as a model/estimator API:

In [ ]:
pipeline.fit(msg_train,label_train)

In [ ]:
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))

Now we have a classification report for our model on a true testing set! There is a lot more to Natural Language Processing than what we've covered here, and its vast expanse of topic could fill up several college courses! I encourage you to check out the resources below for more information on NLP!

## More Resources

Check out the links below for more info on Natural Language Processing:

[NLTK Book Online](http://www.nltk.org/book/)

[Kaggle Walkthrough](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words)

[SciKit Learn's Tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)